In [6]:
import datasets
from collections import Counter
import string
from gensim.utils import tokenize
from torch.utils.data import Dataset

In [8]:
newsdata=datasets.load_dataset('ag_news')

Using the latest cached version of the dataset since ag_news couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/artemybombastic/.cache/huggingface/datasets/ag_news/default/0.0.0/eb185aade064a813bc0b7f42de02595523103ca4 (last modified on Tue Feb 25 16:23:48 2025).


In [21]:
newsdata['train']['label'][0]

2

In [43]:
#Создаём словарь количества вхождений слов
words=[]
#str.maketrans('','',string.punctuation)-возврящяет словарь для замены
#.translate ждёт словарь слов для замены
for sent in newsdata['train']['text']:
    proced_sent=sent.lower().translate(
        str.maketrans('','',string.punctuation)
    )
    for word in tokenize(proced_seq):
        words.append(word)
        

vocab = set(['<unk>','<bos>','<eos>','<pad>'])#Токеный неизвестногого слова, начала,конца последовательности и токен пустного пропуска для батчей


treshold=30 #Порог для включения в словарь word2ind
words=Counter(words)

for word,cnt in words.items():
    if cnt>treshold:
        vocab.add(word)
print(f'Размер словаря {len(vocab)}')
        
   

Размер словаря 34


In [57]:
word2ind={i:char for char,i in enumerate(words)}
ind2word={ca:i for char,i in enumerate(words)}

In [59]:
ind2word

{'nets': 0,
 'get': 1,
 'carter': 2,
 'from': 3,
 'raptors': 4,
 'indianapolis': 5,
 'allstar': 6,
 'vince': 7,
 'was': 8,
 'traded': 9,
 'by': 10,
 'the': 11,
 'toronto': 12,
 'to': 13,
 'new': 14,
 'jersey': 15,
 'for': 16,
 'alonzo': 17,
 'mourning': 18,
 'eric': 19,
 'williams': 20,
 'aaron': 21,
 'and': 22,
 'a': 23,
 'pair': 24,
 'of': 25,
 'firstround': 26,
 'draft': 27,
 'picks': 28,
 'yesterday': 29}

In [33]:
class WordDataset(Dataset):
    def __init__(self,data):  
        super(WordDataset,self).__init__()
        self.data=data
        self.unk='<unk>'
        self.bos='<bos>'
        self.eos='<eos>'
        self.pad='<pad>'        
    def __getitem__(self,idx:int):
        #получаем оригинальные данные
        sent=self.data['text'][idx]
        label=self.data['label'][idx]
        proc_sent=sent.lower().translate(
            str.maketrans('','',string.punctuation)

        )
        #tokenized_sent=tokenize(proc_sent)
        tokenized_sent=[self.bos]
        tokenized_sent+=[
            word2ind.get(word,self.unk) for word in tokenize(proc_sent)
        ]
        tokenized_sent+=[self.eos]

        sample={
            'text':tokenized_sent,
            'label':label
        }
        return sample

In [34]:
Train_dataset=WordDataset(newsdata['train'])



In [56]:
word2ind

{0: 'nets',
 1: 'get',
 2: 'carter',
 3: 'from',
 4: 'raptors',
 5: 'indianapolis',
 6: 'allstar',
 7: 'vince',
 8: 'was',
 9: 'traded',
 10: 'by',
 11: 'the',
 12: 'toronto',
 13: 'to',
 14: 'new',
 15: 'jersey',
 16: 'for',
 17: 'alonzo',
 18: 'mourning',
 19: 'eric',
 20: 'williams',
 21: 'aaron',
 22: 'and',
 23: 'a',
 24: 'pair',
 25: 'of',
 26: 'firstround',
 27: 'draft',
 28: 'picks',
 29: 'yesterday'}

In [54]:
for word in tokenize(newsdata['train']['text'][0]):
    print(word.lower())

wall
st
bears
claw
back
into
the
black
reuters
reuters
short
sellers
wall
street
s
dwindling
band
of
ultra
cynics
are
seeing
green
again


In [39]:
a=tokenize(newsdata['train']['text'][0])

In [41]:
a[0]

TypeError: 'generator' object is not subscriptable

In [37]:
Train_dataset[6]

{'text': ['<bos>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<unk>',
  '<eos>'],
 'label': 2}